In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

# Load the data
data = pd.read_csv('output.csv')

# Drop non-numeric columns
data = data.drop(columns=['Address','Status', 'Data Date'])


In [ ]:
X = data.values[:, :-1]  # All but the last month
y = data.values[:, -1]  # Last month

# Reshape X to be 3D, as expected by LSTMs: (num_samples, num_timesteps, num_features)
X = X.reshape((X.shape[0], X.shape[1], 1))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the LSTM model
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(48, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1)  # Linear activation for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test))

In [ ]:
# Evaluate the model
loss, mae = model.evaluate(X_test, y_test)
print(f'Mean Absolute Error: {mae}')

# Make predictions on the validation data
y_pred = model.predict(X_test)

# If you want to compute R-squared (R²) on validation data
from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_pred)
print(f'R-squared (R²) on validation data: {r2}')

# If you want to predict on the entire dataset
predictions_full = model.predict(X.reshape((X.shape[0], X.shape[1], 1)))  # Ensure X is 3D
predictions_full_df = pd.DataFrame(predictions_full, columns=['Predicted Price'])
print(predictions_full_df)

In [6]:
model.save("saved_model/my_model")

INFO:tensorflow:Assets written to: saved_model/my_model\assets


INFO:tensorflow:Assets written to: saved_model/my_model\assets
